In [ ]:
!pip install PyWavelets

In [ ]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd

In [ ]:
files = glob.glob("shared-nvme/flujos_cepsa/procesa2-2021-0*.csv")

# Read CSV files
# Save it in a CSV or parquet??
dfs = []
for file in files:
    df = pd.read_csv(file)
    dfs.append(df)

# DataFrame with all the data
combined_df = pd.concat(dfs, ignore_index=True)

In [ ]:
wt='gaus1'
scaleMax = 10+1 # 2^10=1024 

# Plot every IP with more than 1000 points
for ip, filtered_df in combined_df.groupby("targetIP"):

    if filtered_df.shape[0] < 1000:
        continue
    
    print(ip)
    xx = filtered_df.sort_values("tref_start", ascending=True)
    X = xx.tref_start
    Y = xx.bpsPhyRcv / xx.bpsPhyRcv.max()
    plt.figure()
    scales = np.power(2, np.arange(scaleMax)) 
    coef, freqs=pywt.cwt(Y, scales, wt)
    
    # Calculate descriptor adding the Y axis
    descriptor = coef.sum(axis=1)
    
    # Save the descriptor matrix in 
    file = f"descriptors/exp/descriptor_{ip}_exp.npy"
    with open(file, 'wb') as f:
        np.save(file, coef)
        
    plt.matshow(coef) 
    plt.title(f"DWT({wt}) {ip}")
    plt.savefig(f"dwt/exp/{ip}_cwt_{wt}_{scaleMax}_plot.pdf")
    plt.close()